# Collecting Data from the Spotify Web API using Spotipy
### About the Spotipy Library:
From the [official Spotipy docs](https://spotipy.readthedocs.io/en/latest/):

"Spotipy is a lightweight Python library for the Spotify Web API. With Spotipy you get full access to all of the music data provided by the Spotify platform."

### About using the Spotify Web API:
Spotify offers a number of [API endpoints](https://developer.spotify.com/documentation/web-api/reference/) to access the Spotify data. In this notebook, I used the following:

- [search endpoint](https://developer.spotify.com/documentation/web-api/reference/search/search/) to get the track IDs
- [audio features endpoint](https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/) to get the corresponding audio features.<br>
The data was collected on several days during the months of April, May and August 2018.

### Goal of this notebook:
The goal is to show how to collect audio features data for tracks from the [official Spotify Web API](https://developer.spotify.com/documentation/web-api/) in order to use it for further analysis/ machine learning which will be part of another notebook.

### 1. Setting Up
The below code is sufficient to set up Spotipy for querying the API endpoint. A more detailed explanation of the whole procedure is available in the [official docs](https://spotipy.readthedocs.io/en/latest/#installation).

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid ="621685b7e9054759ac91dca2a8a0a5f8" 
secret = "3c593b4b86814535a0014c0e6b8ff0dc"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## 2. Get the Track ID Data
The data collection is divided into 2 parts: the track IDs and the audio features. In this step, I'm going to collect 10.000 track IDs from the Spotify API.

The [search endpoint](https://developer.spotify.com/documentation/web-api/reference/search/search/) used in this step had a few limitations:

- limit: a maximum of 50 results can be returned per query
- offset: this is the index of the first result to return, so if you want to get the results with the index 50-100 you will need to set the offset to 50 etc. <br>
Spotify cut down the maximum offset to 10.000.

My solution: using a nested for loop, I increased the offset by 50 in the outer loop until the maxium limit/ offset was reached. The inner for loop did the actual querying and appending the returned results to appropriate lists which I used afterwards to create my dataframe.

In [2]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

for i in range(0,10000,50):
    track_results = sp.search(q='year:2018', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 200.35795813461579


## 3. Data Preparation and EDA
In the next few cells, I'm going to do some exploratory data analysis as well as data preparation of the newly gained data.

A quick check for the track_id list:

In [3]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 10000


Ahh, looks good. Let's get it into a dataframe. 

In [4]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(10000, 4)


,artist_name,track_name,track_id,popularity
0,Travis Scott,SICKO MODE,2xLMifQCjDGFmkHkpNLD9h,96
1,Ariana Grande,"thank u, next",2rPE9A1vEgShuZxxzR2tZH,98
2,Sheck Wes,Mo Bamba,1xzBco0xcoJEDXktl7Jxrr,93
3,Kodak Black,ZEZE (feat. Travis Scott & Offset),7l3E7lcozEodtVsSTCkcaA,95
4,Juice WRLD,Lucid Dreams,0s3nnoMeVWz3989MkNQiRf,94


In [5]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
artist_name    10000 non-null object
track_name     10000 non-null object
track_id       10000 non-null object
popularity     10000 non-null int64
dtypes: int64(1), object(3)
memory usage: 312.6+ KB


** Sometimes, the same track is returned under different track IDs i.e., as asingle track and from album.**

This needs to be checked for and corrected if needed.

In [6]:
# group the entries by artist_name and track_name and check for duplicates

grouped = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped[grouped > 1].count()

2297

There are 2297 duplicate entries which will be dropped in the next cell:

In [7]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [8]:
# doing the same grouping as before to verify the solution
grouped_after_dropping = df_tracks.groupby(['artist_name','track_name'], as_index=True).size()
grouped_after_dropping[grouped_after_dropping > 1].count()

0

In [9]:
# One more way of checking for dupliates
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'],keep=False)].count()

artist_name    0
track_name     0
track_id       0
popularity     0
dtype: int64

In [10]:
df_tracks.shape

(6901, 4)

## 4: Get the Audio Features Data
With the [audio features endpoint](https://developer.spotify.com/documentation/web-api/reference/tracks/get-several-audio-features/) I will now get the audio features data for my 6901 track IDs.

The limitation for this endpoint is that a maximum of 100 track IDs can be submitted per query.

Again, I used a nested for loop. This time the outer loop was pulling track IDs in batches of size 100 and the inner for loop was doing the query and appending the results to the rows list.

Additionaly, I had to implement a check when a track ID didn't return any audio features (i.e. None was returned) as this was causing issues.

In [11]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 94
Time to run this code (in seconds): 21.542112133137834


## 5. Data Preparation
Same as with the first dataset, checking how the rows list looks like:

In [12]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 6807


In [13]:
# Load it into dataframe
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (6807, 18)


,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.00513,https://api.spotify.com/v1/audio-analysis/2xLM...,0.834,312820,0.730,2xLMifQCjDGFmkHkpNLD9h,0.00000,8,0.1240,-3.714,1,0.2220,155.008,4,https://api.spotify.com/v1/tracks/2xLMifQCjDGF...,audio_features,spotify:track:2xLMifQCjDGFmkHkpNLD9h,0.446
1,0.28000,https://api.spotify.com/v1/audio-analysis/2rPE...,0.724,207333,0.647,2rPE9A1vEgShuZxxzR2tZH,0.00000,1,0.1020,-5.642,1,0.0658,106.960,4,https://api.spotify.com/v1/tracks/2rPE9A1vEgSh...,audio_features,spotify:track:2rPE9A1vEgShuZxxzR2tZH,0.435
2,0.19400,https://api.spotify.com/v1/audio-analysis/1xzB...,0.729,183907,0.625,1xzBco0xcoJEDXktl7Jxrr,0.00986,4,0.2480,-5.266,1,0.0315,146.034,4,https://api.spotify.com/v1/tracks/1xzBco0xcoJE...,audio_features,spotify:track:1xzBco0xcoJEDXktl7Jxrr,0.261
3,0.07100,https://api.spotify.com/v1/audio-analysis/7l3E...,0.826,225199,0.615,7l3E7lcozEodtVsSTCkcaA,0.00000,8,0.0965,-7.979,0,0.2190,98.056,4,https://api.spotify.com/v1/tracks/7l3E7lcozEod...,audio_features,spotify:track:7l3E7lcozEodtVsSTCkcaA,0.543
4,0.34900,https://api.spotify.com/v1/audio-analysis/0s3n...,0.511,239836,0.566,0s3nnoMeVWz3989MkNQiRf,0.00000,6,0.3400,-7.230,0,0.2000,83.903,4,https://api.spotify.com/v1/tracks/0s3nnoMeVWz3...,audio_features,spotify:track:0s3nnoMeVWz3989MkNQiRf,0.218


In [14]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6807 entries, 0 to 6806
Data columns (total 18 columns):
acousticness        6807 non-null float64
analysis_url        6807 non-null object
danceability        6807 non-null float64
duration_ms         6807 non-null int64
energy              6807 non-null float64
id                  6807 non-null object
instrumentalness    6807 non-null float64
key                 6807 non-null int64
liveness            6807 non-null float64
loudness            6807 non-null float64
mode                6807 non-null int64
speechiness         6807 non-null float64
tempo               6807 non-null float64
time_signature      6807 non-null int64
track_href          6807 non-null object
type                6807 non-null object
uri                 6807 non-null object
valence             6807 non-null float64
dtypes: float64(9), int64(4), object(5)
memory usage: 957.3+ KB


Some columns are not needed for the analysis so I will drop them.

Also the ID column will be renamed to track_id so that it matches the column name from the first dataframe.

In [15]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(6807, 14)

In [16]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (6807, 14)


,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Travis Scott,SICKO MODE,2xLMifQCjDGFmkHkpNLD9h,96,0.00513,0.834,312820,0.730,0.00000,8,0.1240,-3.714,1,0.2220,155.008,4,0.446
1,Ariana Grande,"thank u, next",2rPE9A1vEgShuZxxzR2tZH,98,0.28000,0.724,207333,0.647,0.00000,1,0.1020,-5.642,1,0.0658,106.960,4,0.435
2,Sheck Wes,Mo Bamba,1xzBco0xcoJEDXktl7Jxrr,93,0.19400,0.729,183907,0.625,0.00986,4,0.2480,-5.266,1,0.0315,146.034,4,0.261
3,Kodak Black,ZEZE (feat. Travis Scott & Offset),7l3E7lcozEodtVsSTCkcaA,95,0.07100,0.826,225199,0.615,0.00000,8,0.0965,-7.979,0,0.2190,98.056,4,0.543
4,Juice WRLD,Lucid Dreams,0s3nnoMeVWz3989MkNQiRf,94,0.34900,0.511,239836,0.566,0.00000,6,0.3400,-7.230,0,0.2000,83.903,4,0.218


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6807 entries, 0 to 6806
Data columns (total 17 columns):
artist_name         6807 non-null object
track_name          6807 non-null object
track_id            6807 non-null object
popularity          6807 non-null int64
acousticness        6807 non-null float64
danceability        6807 non-null float64
duration_ms         6807 non-null int64
energy              6807 non-null float64
instrumentalness    6807 non-null float64
key                 6807 non-null int64
liveness            6807 non-null float64
loudness            6807 non-null float64
mode                6807 non-null int64
speechiness         6807 non-null float64
tempo               6807 non-null float64
time_signature      6807 non-null int64
valence             6807 non-null float64
dtypes: float64(9), int64(5), object(3)
memory usage: 957.2+ KB


In [18]:
# Check for dulpicates
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence


Everything seems to be fine so I will save the dataframe as a .csv file.

In [19]:
df.to_csv('SpotifyAudioFeatures11152018.csv')